In [ ]:
import numpy as np
import matplotlib.pylab as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_probability as tfp
tfd = tfp.distributions
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from tensorflow.keras.models import load_model, Sequential, Model

np.random.seed(12211)  

In [ ]:
num_train = 200000
num_test = 20000

In [ ]:
Trainset = ['FSPSlin', 'FSPSlog', 'FSPSall', 'OBS', 'UM', 'BP', 'UMnew'][6]#6
Testset = ['FSPSlin', 'FSPSlog', 'FSPSall', 'OBS', 'UM', 'BP', 'UMnew'][3]

In [ ]:
n_epochs = 2000 #20
D = 5 
K = 3

learning_rate = 1e-5
decay_rate= 1e-3 
batch_size = 1024 

save_mod = 'saved_hubs/tf2models/'+'Train_'+Trainset+'_lr_'+str(learning_rate)+'_dr'+str(decay_rate)+'_ne'+str(n_epochs)+'_k'+str(K)+'_nt'+str(num_train)

In [ ]:
def minmax_cutsOBSarr(X, y, l):
    # print(X.shape)
    
    mask_cond =  np.where( 
        (X[:, 0] < max_col[0]) & (X[:, 0] > min_col[0]) &
        (X[:, 1] < max_col[1]) & (X[:, 1] > min_col[1]) &
        (X[:, 2] < max_col[2]) & (X[:, 2] > min_col[2]) &
        (X[:, 3] < max_col[3]) & (X[:, 3] > min_col[3]) & 
        (X[:, 4] < max_mag) & (X[:, 4] > min_mag) &
        (y < max_z) & (y > min_z) )
    
    # print( np.array(mask_cond).shape)

    X_new = X[mask_cond]
    y_new = y[mask_cond]
    l_new = l[mask_cond]
    # print(X_new.shape)
    return X_new, y_new, l_new, mask_cond


In [ ]:
def print_limits(X, y):
    print(10*'-')
    print('number of datapoints: ', str(y.shape[0]))
    print('z-minmax: ', y.min(), y.max())
    print('ColMag-min: ', np.min(X, axis=0))
    print('ColMag-max: ', np.max(X, axis=0))
    print(10*'-')

In [ ]:
def shuffle(X, y):
    shuffleOrder = np.arange(X.shape[0])
    np.random.shuffle(shuffleOrder)
    X = X[shuffleOrder]
    y = y[shuffleOrder]
    return X, y, shuffleOrder

In [ ]:
def loadTrainTest(dirIn = '../../Data/fromGalaxev/photozs/datasets/data_may_2020/'):
    
    test_data = np.load(dirIn + 'test_' + Testset +'.npy') 

    X_test = test_data[: , :-1]
    y_test = test_data[: , -1]

    print_limits(X_test, y_test)

    if Testset == 'OBS':
        test_labels = np.load(dirIn + 'test_' + Testset + '_label.npy') 

    return _, _, X_test, y_test, test_labels


In [ ]:
_, _, X_test, y_test, label_test = loadTrainTest(dirIn = 'Data/fromGalaxev/photozs/datasets/data_may_2020/')

In [ ]:
min_col = [-0.09145837, -0.05327791, -0.02479261, -0.10519464] #-0.03 #-5
max_col = [ 3.825315,   2.8303378,  1.6937237,  1.5019817] #3.4 #5
min_mag = 12
max_mag = 23
min_z = 0.0 #np.min(y_train) 
max_z = 1.1 #np.max(y_train) 


X_test, y_test, label_test, mask_cond = minmax_cutsOBSarr(X_test, y_test, label_test)

In [ ]:
print("Size of features in test data: {}".format(X_test.shape))
print("Size of output in test data: {}".format(y_test.shape))

In [ ]:

preproc = Pipeline([('stdscaler', StandardScaler())])
scalerfile = save_mod + '_scaling_X'
preproc = pickle.load(open(scalerfile, 'rb'))
X_test = preproc.transform(X_test)


preproc_y = Pipeline([('stdscaler', MinMaxScaler())])
scalerfile = save_mod + '_scaling_y'
preproc_y = pickle.load(open(scalerfile, 'rb'))


y_test = preproc_y.transform(y_test.reshape(-1, 1))

In [ ]:
plt.figure(23)

plt.hist(y_test, density=True, bins = 250, histtype='step', label='test')
plt.legend()

In [ ]:
non_lin_act = tf.nn.relu #tf.nn.tanh
y_true = tf.keras.Input(shape=(1,))
inputs = tf.keras.Input(shape=(D,))
layer_1 = tf.keras.layers.Dense(units=512, activation=non_lin_act)(inputs)
layer_1a = tf.keras.layers.Dense(units=1024, activation=non_lin_act)(layer_1)
layer_1b = tf.keras.layers.Dense(units=2048, activation=non_lin_act)(layer_1a)
layer_1c = tf.keras.layers.Dense(units=1024, activation=non_lin_act)(layer_1b)
layer_2 = tf.keras.layers.Dense(units=512, activation=non_lin_act)(layer_1c)
layer_3 = tf.keras.layers.Dense(units=256, activation=non_lin_act)(layer_2)
layer_4 = tf.keras.layers.Dense(units=128, activation=non_lin_act)(layer_3)
layer_5 = tf.keras.layers.Dense(units=64, activation=non_lin_act)(layer_4)
layer_6 = tf.keras.layers.Dense(units=32, activation=non_lin_act)(layer_5)
mu = tf.keras.layers.Dense(units=K, activation=None, name="mu")(layer_6)
var = tf.keras.backend.exp(tf.keras.layers.Dense(units=K, activation=tf.nn.softplus, name="sigma")(layer_6))
pi = tf.keras.layers.Dense(units=K, activation=tf.nn.softmax, name="mixing")(layer_6)

model_train = Model([inputs, y_true], [mu, var, pi], name='mdn')

In [ ]:
# Define custom loss
def custom_loss(layer):
    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
    def loss(y_true, mu, var, pi):
        mixture_distribution = tfp.distributions.Categorical(probs=pi)
        distribution = tfp.distributions.Normal(loc=mu, scale=var)
        likelihood = tfp.distributions.MixtureSameFamily(mixture_distribution=mixture_distribution,components_distribution=distribution)

        log_likelihood = -1.0*likelihood.log_prob(tf.transpose(y_true))
        mean_loss = tf.reduce_mean(log_likelihood)

        return mean_loss
    return loss
    
# Compile the model
model_train.add_loss(custom_loss(inputs)(y_true, mu, var, pi))


model_train.compile(optimizer='adam')
model_train.summary()

In [ ]:
model_train.load_weights(save_mod + '.h5')

y_pred = np.array(model_train(X_test))

In [ ]:
y_pred_arg = np.argmax(y_pred[2, :, :], axis = 1)
y_pred_mean = y_pred[0, :, :][:, y_pred_arg][:, 0]
y_pred_std = np.sqrt(np.log(y_pred[1, :, :][:, y_pred_arg][:, 0]))

In [ ]:
y_pred_3means = preproc_y.inverse_transform(y_pred[0, :, :])
y_pred_3std = preproc_y.inverse_transform( np.sqrt(np.log(y_pred[1, :, :])  ))
y_pred_3weights = y_pred[2, :, :]

y_test_all = preproc_y.inverse_transform(y_test)


predstdweights = np.array([y_pred_3means, y_pred_3std, y_pred_3weights])
truelabel = np.array([y_test_all[:, 0], label_test])

In [ ]:

ifPlotWeighted = True
y_pred_mean_best = y_pred_mean
y_pred_std_best = y_pred_std


if ifPlotWeighted:
    plt.figure(22, figsize=(10, 10))


    plt.errorbar(preproc_y.inverse_transform(y_test)[:, 0], preproc_y.inverse_transform(y_pred_mean_best.reshape(-1, 1))[:, 0], yerr= preproc_y.inverse_transform(y_pred_std_best.reshape(-1, 1) )[:, 0], fmt='ro', ecolor='k', ms = 5, alpha = 0.1, label = 'Training with synthetic data')
    



plt.plot([0, 1], [0, 1], 'k')
plt.plot([0, 1], 0.85*np.array([0, 1]), 'k-.')
plt.plot([0, 1], 1.15*np.array([0, 1]), 'k-.')


plt.ylabel(r'Photometric redshift', fontsize=25)
plt.xlabel(r'True redshift', fontsize=25)

plt.tight_layout()

plt.axes().set_aspect('equal')


leg = plt.legend(fontsize = 'xx-large', markerscale=1., numpoints=2)

for artist, text in zip(leg.legendHandles, leg.get_texts()):
    col = artist.get_color()
    if isinstance(col, np.ndarray):
        col = col[0]
    text.set_color(col)
    text.set_alpha(1.0)


# plt.savefig('phoz_compare.pdf', bbox_inches='tight')

plt.show()

In [ ]:


ifPlotWeighted = True

if ifPlotWeighted:
    

    colorstring = ['b', 'r', 'g', 'k', 'orange']
    surveystring = ['Primus', 'Vipers', 'SDSS', 'DEEP2', 'Wiggle-z']


    for label_ind in [0, 1, 2, 3, 4]:

        plt.figure(22, figsize=(10, 10,))

        surveyindx = np.where(label_test == label_ind)

        offset = 0.04
        
        plt.errorbar(preproc_y.inverse_transform(y_test)[surveyindx][:, 0], offset + preproc_y.inverse_transform(y_pred_mean_best.reshape(-1, 1))[surveyindx][:, 0], yerr= preproc_y.inverse_transform(y_pred_std_best.reshape(-1, 1))[surveyindx][:, 0], fmt = 'o', marker=None, ms = 4, alpha = 0.3, label = 'Training: Synthetic, Testing: '+surveystring[label_ind], c = colorstring[label_ind])


        C = 0.05
        z_t = np.array([0, 1])
        z_tp = z_t + C*(1+z_t)
        z_tm = z_t - C*(1+z_t)

        plt.plot(z_t, z_t, 'k')



        plt.plot(z_t, z_tp, 'k-.')
        plt.plot(z_t, z_tm, 'k-.')




        plt.ylabel(r'$z_{phot}$', fontsize=25)
        plt.xlabel(r'$z_{spec}$', fontsize=25)
        
        plt.xlim(0.0, 1)
        plt.ylim(0.0, 1)


        plt.tight_layout()

        plt.axes().set_aspect('equal')


        leg = plt.legend(fontsize = 'xx-large', markerscale=1., numpoints=2)

        trueprederr = np.array([preproc_y.inverse_transform(y_test)[surveyindx], offset + (preproc_y.inverse_transform(y_pred_mean_best.reshape(-1, 1))[surveyindx]), preproc_y.inverse_transform(y_pred_std_best.reshape(-1, 1))[surveyindx]])
#         np.save(save_mod + 'test_label_' +str(label_ind), trueprederr)



# plt.savefig('phoz_compare_surveys.pdf', bbox_inches='tight')

plt.show()
